In [1]:
OUT_DIR = '/tmp/'

from torchvision import models
import torchvision.datasets as dsets
import json
import timm
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from robustness import model_utils, datasets, train, defaults
from robustness.datasets import CIFAR, ImageNet
import torch
from cox.utils import Parameters
import cox.store
from torchvision import models
import torch
from torchvision import transforms
from torchvision import models
import numpy as np
import matplotlib.pyplot as plt
# Standard libraries
import numpy as np
import os
# PyTorch
import torch.nn as nn
from PIL import Image

/home/hashmat.malik/.conda/envs/advdrop_env/lib/python3.7/site-packages/robustness/train.py:24: UserWarning: Could not import amp.
  warnings.warn('Could not import amp.')


In [2]:

class_idx = json.load(open("/home/hashmat.malik/PycharmProjects/Thesis/AdvDrop/imagenet_class_index.json"))
idx2label = [class_idx[str(k)][1] for k in range(len(class_idx))]
class2label = [class_idx[str(k)][0] for k in range(len(class_idx))]

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(), ])


def image_folder_custom_label(root, transform, idx2label) :
    
    
    old_data = dsets.ImageFolder(root=root, transform=transform)
    old_classes = old_data.classes
    
    label2idx = {}
    
    for i, item in enumerate(idx2label) :
        label2idx[item] = i
    
    new_data = dsets.ImageFolder(root=root, transform=transform, 
                                 target_transform=lambda x : idx2label.index(old_classes[x]))
    new_data.classes = idx2label
    new_data.class_to_idx = label2idx

    return new_data

def create_dir(dir, print_flag = False):
    if not os.path.exists(dir):
        os.mkdir(dir)
        if print_flag:
            print("Create dir {} successfully!".format(dir))
    elif print_flag:
        print("Directory {} is already existed. ".format(dir))


def get_data(data_dir, transform, batch_size, class2label):

    normal_data = image_folder_custom_label(root=data_dir,
                                            transform=transform,
                                            idx2label=class2label)
    normal_loader = torch.utils.data.DataLoader(normal_data,
                                                batch_size=batch_size,
                                                shuffle=False)

    return normal_loader

## Load VGG Models

In [3]:
imagenet_ds = ImageNet('/tmp/')
vgg , _ = model_utils.make_and_restore_model(arch=models.vgg16_bn(), dataset=imagenet_ds, 
                                resume_path='/home/hashmat.malik/PycharmProjects/Thesis/AdvDrop/pretrained-models/vgg16_bn_l2_eps0.ckpt',
                                parallel=False, add_custom_forward=True)
model = vgg.model

=> loading checkpoint '/home/hashmat/Thesis/AdvDrop/pretrained-models/vgg16_bn_l2_eps0.ckpt'
=> loaded checkpoint '/home/hashmat/Thesis/AdvDrop/pretrained-models/vgg16_bn_l2_eps0.ckpt' (epoch 89)


## Load MobileNet-v2

In [ ]:
imagenet_ds = ImageNet('/tmp/')
mobilenet , _ = model_utils.make_and_restore_model(arch=models.mobilenet_v2(), dataset=imagenet_ds, 
                                resume_path='/home/hashmat.malik/PycharmProjects/Thesis/AdvDrop/pretrained-models/mobilenet_l2_eps0.ckpt',
                                parallel=False, add_custom_forward=True)
model = mobilenet.model

## Load ResNet Models

In [3]:
imagenet_ds = ImageNet('/tmp/')
resnet , _ = model_utils.make_and_restore_model(arch="resnet18", dataset=imagenet_ds, 
                                resume_path='/home/hashmat.malik/PycharmProjects/Thesis/AdvDrop/pretrained-models/resnet18_l2_eps0.ckpt',
                                parallel=False, add_custom_forward=True)
model = resnet.model

=> loading checkpoint '/home/hashmat.malik/PycharmProjects/Thesis/AdvDrop/pretrained-models/resnet18_l2_eps0.ckpt'
=> loaded checkpoint '/home/hashmat.malik/PycharmProjects/Thesis/AdvDrop/pretrained-models/resnet18_l2_eps0.ckpt' (epoch 90)


In [ ]:
#from Models.transformers import diet_tiny, diet_small, vit_tiny, vit_small
class Normalize(nn.Module):
    def __init__(self, mean, std):
        super(Normalize, self).__init__()
        self.register_buffer('mean', torch.Tensor(mean))
        self.register_buffer('std', torch.Tensor(std))

    def forward(self, input):
        # Broadcasting
        input = input / 255.0
        mean = self.mean.reshape(1, 3, 1, 1)
        std = self.std.reshape(1, 3, 1, 1)
        return (input - mean) / std

transform = transforms.Compose([
transforms.Resize((224, 224)),
transforms.ToTensor(),
transforms.Normalize(mean=[0.485, 0.456, 0.406],
                   std=[0.229, 0.224, 0.225])])


def evaluate_accuracy(model, bs=128, 
                 data_path="/home/hashmat.malik/PycharmProjects/Thesis/nobox-attacks/data/ILSVRC2012_img_val",
                     tf = transform):

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    OUT_DIR = '/tmp/'
    imagenet_ds = ImageNet('/tmp/')
    model = model.eval()
    batch_size = bs
    data_dir = data_path
    normal_loader = get_data(data_dir, tf, batch_size, class2label)
    correct = 0
    total = 0
    count = 0
    for images, labels in normal_loader:
        images, labels = images.to(device), labels.to(device)
        preds = model(images)
        _, pre = torch.max(preds, 1)
        correct += torch.sum(pre==labels)
        total += images.shape[0]
        print(total)
    print(f"The Model accuracy is {correct/total}")



   


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
